In [1]:
import torch
from model import *
from model2 import Encoder_decoder
import model
from utils import Combined_Loss, Combined_Loss2
from dataloaders. import VAEDataset, VAEDataset1
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchsummary import summary
import time
from torchvision.models import resnet50, ResNet50_Weights
resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2).cuda()

SyntaxError: invalid syntax (<ipython-input-1-a00d808e3ac7>, line 6)

In [ ]:
HALF = False
PIN_MEM = True
BATCH_SIZE_TRAIN = 1
BATCH_SIZE_TEST = 1
NUM_WORKERS = 0
LOAD = True
DEPTH = 5
train_save = open("log/train_loss.txt", "a")
test_save = open("log/test_loss.txt", "a")

vae = Encoder_decoder(DEPTH).cuda()
#t = vae.train()
if LOAD:
    vae.load_state_dict(torch.load("w2.pth"))
    t = vae.eval()
else:
    t = vae.train()
#rgb_p = 'archive/FLIR_ADAS_1_3/train/RGB/'
#ir__p = 'archive/FLIR_ADAS_1_3/train/thermal_8_bit/'

rgb_p = 'archive/dataset/files/img_raw/rgb/'
ir__p = 'archive/dataset/files/img_raw/nir/'

train_ds = VAEDataset1(rgb_p, ir__p, half=HALF)
train_loader = DataLoader(
        dataset=train_ds,
        batch_size=BATCH_SIZE_TRAIN,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEM,
        shuffle=True
    )

#rgb_vp = 'archive/FLIR_ADAS_1_3/val/RGB/'
#ir__vp = 'archive/FLIR_ADAS_1_3/val/thermal_8_bit/'
rgb_vp = 'archive/dataset/files/img_raw/rgb/'
ir__vp = 'archive/dataset/files/img_raw/nir/'

val_ds = VAEDataset1(rgb_vp, ir__vp, half=HALF)
val_loader = DataLoader(
        dataset=val_ds,
        batch_size=BATCH_SIZE_TEST,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEM,
        shuffle=True
    )


optimizer = torch.optim.Adam(vae.parameters(), lr=0.0001)
loss = Combined_Loss()
loss2 = Combined_Loss2()
#summary(vae, [(1,256,256), (3,256,256)])


In [ ]:
def pyramid(ir, rgb):
    ir, rgb = [ir.cuda()], [rgb.cuda()]
    for i in range(1, DEPTH):
        ir.append(nn.functional.interpolate(ir[-1], scale_factor=0.5).cuda())
        rgb.append(nn.functional.interpolate(rgb[-1], scale_factor=0.5).cuda())
    return ir, rgb

In [2]:
for epc in range(0,200):
    epoch_loss = []
    r = tqdm(enumerate(train_loader), leave=False, desc="Epoch: {}".format(epc), total=len(train_loader))
    for idx, (ir, rgb) in r:
        ir, rgb = pyramid(ir, rgb)
        If = vae(rgb, ir)
        lf = loss2.compute(ir[0], rgb[0], If)
        epoch_loss.append(lf.item())
        del rgb, ir, If
        torch.cuda.empty_cache()
        optimizer.zero_grad()
        lf.backward()
        torch.nn.utils.clip_grad_norm_(vae.parameters(), max_norm=1.0, norm_type=1)
        optimizer.step()
        r.set_postfix(loss=lf.item())
    #save epoch loss list
    train_save.write("epoch [{}] = ".format(epc) + str(epoch_loss) + '\n')
    #save checkpoint
    torch.save(vae.state_dict(), "w2.pth")
    
    #test loss run on val
    with torch.no_grad():
        e = vae.eval()
        ir, rgb = next(iter(val_loader))
        ir, rgb = pyramid(ir, rgb)
        If = vae(rgb, ir)
        lf = loss2.compute(ir[0], rgb[0], If)
        print("epoch [{}] = ".format(epc) + str(float(lf)))
        test_save.write("epoch [{}] = ".format(epc) + str(float(lf)) + '\n')
        
        del rgb, ir, If
        torch.cuda.empty_cache()

NameError: name 'tqdm' is not defined

In [ ]:
for epc in range(0,200):
    epoch_loss = []
    r = tqdm(enumerate(train_loader), leave=False, desc="Epoch: {}".format(epc), total=len(train_loader))
    for idx, (ir, rgb) in r:
        ir, rgb = ir.cuda(), rgb.cuda()
        If, mu, sigma = vae(ir, rgb)
        lf = loss.compute(ir, rgb, mu, sigma, If)
        epoch_loss.append(lf.item())
        del rgb, ir, If, mu, sigma
        torch.cuda.empty_cache()
        optimizer.zero_grad()
        lf.backward()
        torch.nn.utils.clip_grad_norm_(vae.parameters(), max_norm=1.0, norm_type=1)
        optimizer.step()
        r.set_postfix(loss=lf.item())
        #time.sleep(0.1)
    #save epoch loss list
    train_save.write("epoch [{}] = ".format(epc) + str(epoch_loss) + '\n')
    #save checkpoint
    torch.save(vae.state_dict(), "w2.pth")
    
    #test loss run on val
    with torch.no_grad():
        e = vae.eval()
        ir, rgb = next(iter(val_loader))
        ir = ir.cuda()
        rgb = rgb.cuda()
        If, mu, sigma = vae(ir, rgb)
        lf = loss.compute(ir, rgb, mu, sigma, If)
        print("epoch [{}] = ".format(epc) + str(float(lf)))
        test_save.write("epoch [{}] = ".format(epc) + str(float(lf)) + '\n')
        
        del rgb, ir, If, mu, sigma, lf
        torch.cuda.empty_cache()